# Practical 1: Processing ground-based gravity data

In this practical, we'll process a set of synthetic gravity surveys near the city of Cape Town, South Africa. The surveys were conducted with a LaCoste & Romberg Model G gravimeter and tied to a single base station (station 0). The synthetic data were generated based on public domain data for Southern Africa distributed by the [NOAA National Centers for Environmental Information](https://www.ngdc.noaa.gov/).

## Goals

* Process the survey data from raw readings into absolute gravity
* Merge them into a single database

## Data 

For this practical, you'll need:

* A zip archive with the survey data: [`cape-town-surveys.zip`](https://github.com/leouieda/gravity-processing/raw/main/data/cape-town-surveys.zip)
* The gravimeter calibration table to convert readings to mGal: [`gravimeter-scaling.csv`](https://raw.githubusercontent.com/leouieda/gravity-processing/main/data/gravimeter-scaling.csv)

Download both files, unzip the archive, and place everything into the **same folder as this notebook**. Your should folder structure should look like this:

```
gravity-practicals/
├── cape-town-surveys/
│   ├── cape-town-gravity-day-1.csv
│   ├── cape-town-gravity-day-2.csv
│   ├── cape-town-gravity-day-3.csv
│   ├── cape-town-gravity-day-4.csv
│   └── cape-town-gravity-day-5.csv
├── gravimeter-scaling.csv
└── practical1.ipynb
```

### Survey data

The survey files are in Comma Separated Values (CSV) format, which is a table of values with rows representing individual survey points and columns representing the position, time, and meter reading. For example, these are the first few lines of one of these files:

```
station_id,longitude,latitude,easting,northing,elevation,time_minutes,reading
0,18.34444,-34.12971,255105.43,6220276.33,32.2,0.0,2555.08
1,18.37418,-34.19583,258037.64,6213013.01,18.4,8.0,2565.12
2,18.40388,-34.23972,260899.25,6208214.71,25.0,14.0,2569.49
3,18.41112,-34.16444,261353.99,6216582.03,228.7,23.0,2514.59
```

* Station ID is a unique identifier (across all surveys) for each survey point. 
* Latitude and longitude are geographic coordinates in degress referenced to the WGS84 ellipsoid.
* Easting and northing are UTM coordinates in meters.
* Elevation is the height of gravimeter above the WGS84 ellipsoid in meters.
* Time are the minutes since the first reading at the base station.
* Reading is the gravimeter reading (unscaled) at that stattion.

The surveys were conducted in a loop, beginning and ending at the base station.

### Gravimeter calibration table

The gravimeter calibration table is particular to the gravimeter used for this survey. The table was provided by the manufacturer. This is what the first few lines of the table look like:

```
counter_reading,value_mgal,interval_factor
0000,0000.00,1.00636
0100,0100.64,1.00621
0200,0201.26,1.00609
0300,0301.87,1.00597
0400,0402.46,1.00588
0500,0503.05,1.00579
0600,0603.63,1.00570
0700,0704.20,1.00563
```

Convert the reading to mGal using the table values and the following equation:

```
reading_mgal = (reading - counter_reading) * interval_factor + value_mgal
```

### Base station

The base station for all surveys was the same. Here are it's properties:

* Station ID: 0
* Absolute gravity: 979656.12 mGal

## Survey location

The surveys were caried out near Cape Town, South Africa. Most points fall around False Bay and the Cape Peninsula, stretching a bit northwards. See the Wikipedia page on the [Geology of Cape Town](https://en.wikipedia.org/wiki/Geology_of_Cape_Town) for details on the geologic context.

Here is a cool thing you can do in notebooks: embed an interactive Google Map (or any HTML page, actually)! To do this, get the HTML code to embed a map by clicking on the "share" button on Google Maps. Copy the `src` part of the code and use `IPython.display.IFrame` to insert it into the notebook.

In [ ]:
from IPython.display import IFrame

In [ ]:
IFrame(src="https://www.google.com/maps/d/embed?mid=1FunoJz1_shZ3wxfNFpg5FH33qkOg3tie", width=800, height=600)

## Import the required libraries

To deal with this type of tabular data, all we need are numpy and matplotlib.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Reading CSV data with numpy

To import the CSV data into a numpy array, we'll use the `numpy.loadtxt` function. By default, it expects spaces between values instead of commas, so we have to use the `delimiter` argument to specify that it needs to use commas. We also need to skip the first line of the file that includes the header (names of each column) using the `skiprows` argument.

----

## Your turn

Create variables called `station_id`, `easting`, `northing`, `time`, and `reading` with the respective data values (columns in the `data` array).

Tip: Use array slicing (like `variable[5:10, 7:8]`) to separate each column of the array.


----

## Plot the survey locations on a map

Now that we have the easting and northing coordinates of each point, we can plot them on a map to see what kind of layout this survey had.

## Drift correction

First thing we need to do is correct for the drift. Let's plot the first and last readings to see how much drift there was.

Assuming that the drift is linear, we can calculate the drift factor $\alpha = \dfrac{\Delta r}{\Delta t}$

----

### Your turn

Apply the drift correction to the readings ($r_{corrected} = r - \alpha\ t$) and store the results in a variable called `reading_nodrift`. Make a plot of the raw readings and the corrected readings as a function of time (reading in y-axis, time in x-axis).

----

## Convert readings to mGal

The next step is to convert the drif-corrected readings to mGal using the conversion table for this gravimeter. First, we have to load the conversion table using numpy.

To illustrate how this is done, let's convert the first reading to mGal. We'll use the values in the table and the formula given above for the conversion.

The trick is figuring out the position (row) in the table that has the values we want for a particular reading. The reading has to be within the counter interval for that row. For example, a reading of 2710 would use the values in row 27 of the table. The position/row can be calculated using the integer division (ignoring the remainder and decimal places) of the reading by 100. With this, we can get the exact conversion factor and value in mGal from the table.

Now that we know how to convert one reading, we can repeat this for all readings using a `for` loop. We can use the `range` function to generate a list of values from 0 to a given number. These will be the position of the readings in our array.

## Calculate absolute gravity

Now that we have our relative readings in mGal, we can tie them to the base station and calculate absolute gravity:

$$ g_{station} = (r_{station} - r_{base}) + g_{base} $$

The first thing we need to do is calculate the difference between the readings and the base station (station 0).

Now we can plot the absolute gravity values on a map using the `plt.scatter` function.

----

## Your turn

Repeat the calculations above for every survey in the zip archive. Use a `for` loop to process all surveys instead of copy/pasting code. Combine the arrays for coordinates, elevation, and absolute gravity from each survey into arrays with all data from all surveys. Make a map of the absolute gravity values of the entire dataset.

Tip: Use the `glob` module to get a list of all file names in a folder.

In [ ]:
import glob

In [ ]:
# The * is a special character that means "any character in any amount"
survey_files = glob.glob("cape-town-surveys/*.csv")
print(survey_files)

Start with empty lists for the coordinates, readings, etc. Then store the arrays for each survey in these lists using `gravities.append(gravity)`, for example. We can then join them together into single arrays using `numpy.concatenate`.

In [ ]:
latitudes = []
longitudes = []
eastings = []
northings = []
elevations = []
gravities = []

In [ ]:
longitude = np.concatenate(longitudes)
latitude = np.concatenate(latitudes)
easting = np.concatenate(eastings)
northing = np.concatenate(northings)
elevation = np.concatenate(elevations)
gravity = np.concatenate(gravities)